In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-chunks/gnostic_chunks.json
/kaggle/input/text-chunks/bible_chunks.json


In [109]:
import json

with open("/kaggle/input/text-chunks/bible_chunks.json","r") as f:
    bible = json.load(f)

with open("/kaggle/input/text-chunks/gnostic_chunks.json","r") as f:
    gnostic = json.load(f)

In [4]:
mat = bible['Matthew']
got = gnostic['Gospel of Thomas']

In [3]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer("all-MiniLM-L6-v2",device='cuda')
def embed(text):
    embedding = model.encode(text,show_progress=True, convert_to_numpy=True)
    return embedding

2025-04-20 06:26:33.960260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745130394.148889      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745130394.205221      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
b = []
for k in bible.keys():
    b.extend(bible[k])

In [7]:
g = []
for k in gnostic.keys():
    g.extend(gnostic[k])

In [9]:
bemb = embed(b)
gemb = embed(g)

Batches:   0%|          | 0/198 [00:00<?, ?it/s]

Batches:   0%|          | 0/262 [00:00<?, ?it/s]

In [11]:
cos = util.cos_sim(gemb,bemb)

In [13]:
print(gemb.shape,bemb.shape,cos.shape)

(8373, 384) (6307, 384) torch.Size([8373, 6307])


In [24]:
ri,ci = np.where(cos>0.5)

In [27]:
pairs = []
for r,c in zip(ri,ci):
    score = cos[r,c]
    gt = g[r]
    bt = b[c]
    pairs.append((gt,bt,score.item()))

In [6]:
import spacy
import matplotlib.pyplot as plt
import re

nlp = spacy.load("en_core_web_sm")

def extract_entities_and_relations(paragraphs):
    """
    Extract cosmological entities and relationships using NER and dependency parsing.
    Args:
        paragraphs (list): List of text paragraphs.
    Returns:
        tuple: Lists of entities and relationships.
    """
    entities = set()
    relationships = []
    
    for para in paragraphs:
        doc = nlp(para)
        # Extract entities (customize for Gnostic terms)
        for ent in doc.ents:
            if ent.label_ in ["DIVINE", "NORP", "REALM"]:  # Adjust for divine beings, realms
                entities.add(ent.text)
        
        # Extract relationships (simplified example)
        for sent in doc.sents:
            for token in sent:
                if token.lemma_ in ["emanate", "beget", "create", "produce"]:
                    # Look for subject and object
                    subject = None
                    obj = None
                    for child in token.children:
                        if child.dep_ in ["nsubj", "nsubjpass"]:
                            subject = child.text
                        if child.dep_ in ["dobj", "attr"]:
                            obj = child.text
                    if subject and obj:
                        relationships.append((subject, obj, token.lemma_))
    
    return list(entities), relationships

In [38]:
ow = gnostic['On the Origin of the World']

In [8]:
en,rl = extract_entities_and_relations(g)

In [14]:
doc= nlp("".join(g))

In [17]:
nc = [chunks.text for chunks in doc.noun_chunks]

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
vect = TfidfVectorizer(ngram_range = (1,3),stop_words='english')
x=vect.fit_transform(nc)

In [79]:
fn=vect.get_feature_names_out()

In [80]:
len(fn)

7923

In [81]:
scores = x.sum(axis=0).A1

In [85]:
top=scores.argsort()[-20:]

In [86]:
tp = [fn[i] for i in top]

In [87]:
tp

['soul',
 'order',
 'powers',
 'glory',
 'word',
 'lines',
 'truth',
 'aeons',
 'life',
 'spirit',
 'knowledge',
 'world',
 'son',
 'place',
 'power',
 'man',
 'light',
 'god',
 'things',
 'father']

## Entity extraction bookwise

In [110]:
from collections import defaultdict

In [ ]:
!python -m spacy download en_core_web_lg

In [116]:
nlp = spacy.load("en_core_web_lg")

In [125]:
entities = defaultdict(lambda : defaultdict(list))

for k,v in gnostic.items():
    for text in v:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ not in ['ORDINAL','CARDINAL']:
                entities[k][(ent.label_,ent.text)].append(text)

In [118]:
entities.keys()

dict_keys(['Eugnostos the Blessed', 'The Letter of Peter to Philip', 'The Second Treatise of the Great Seth', 'Authoritative Teaching', 'The Interpretation of Knowledge', 'The Thought of Norea', 'Melchizedek', 'The Apocalypse of Adam', 'The Dialogue of the Savior', 'The Hypostasis of The Archons', 'The Tripartite Tractate', 'The Apocryphon of James', 'second_apocalypse_of_james_verses', 'The Apocryphon of John', 'The Three Steles of Seth', 'The Sentences of Sextus', 'On the Baptism A', 'The Treatise on the Resurrection', 'The (First) Apocalypse of James', 'Allogenes', 'Zostrianos', 'The Testimony of Truth', 'The Sophia of Jesus Christ', 'A Valentinian Exposition', 'The Prayer of the Apostle Paul', 'On the Anointing', 'Marsanes', 'The Thunder Perfect Mind', 'The Gospel of Philip', 'The Concept of Our Great Power', 'Plato Republic 588A-589B', 'Trimorphic Protennoia', 'The Apocalypse of Paul', 'Asclepius 21-29', 'On the Origin of the World', 'Gospel of Thomas', 'The Discourse on the Eight

In [119]:
entities['The Tripartite Tractate']

defaultdict(list,
            {'Jerusalem': ['And he spoke to him, saying, "By which road shall I go up to Jerusalem?"',
              'For I have come to you that you may go up to Jerusalem to your fellow apostles.'],
             'Paul': ['The little child knew who Paul was.',
              'The little child spoke, saying, "I know who you are, Paul.',
              'Let your mind awaken, Paul, with [...].',
              'And after he brought that speech to an end, he spoke, saying to me, "Let your mind awaken, Paul, and see that this mountain upon which you are standing is the mountain of Jericho, so that you may know the hidden things in those that are visible.',
              'Then I gazed upward and saw the Spirit saying to me, "Paul, come!',
              'The old man spoke, saying to me, "Where are you going, Paul?'],
             'Spirit': ['And I am the Spirit who accompanies you.',
              'Then he gazed down and saw the twelve apostles at his right and at his left in 

In [120]:
output = []

for book, entity in entities.items():
    for en,text in entity.items():
        output.append(
            {
                "book": book,
                "entity":en,
                "context":text,
                "count":len(text)
            }
        )

In [ ]:
!pip install spacy-llm

In [100]:
t = "".join(gnostic['Gospel of Thomas'])

In [102]:
import spacy

nlp = spacy.blank("en")
llm = nlp.add_pipe("llm_textcat")
llm.add_label("DIVINE")
llm.add_label("REALM")
doc = nlp(t)
print(doc.cats)
# {"COMPLIMENT": 1.0, "INSULT": 0.0}

ValueError: [E002] Can't find factory for 'llm_textcat' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, en.lemmatizer